In [1]:
import cv2 as cv
import time
import numpy as np

In [2]:
cv.__version__

'4.6.0'

In [ ]:
def extract_frames(path):
    video = cv.VideoCapture(path)
    num_rames = int(video.get(cv.CAP_PROP_FRAME_COUNT))
    frames = [None] * num_rames
    for i in range(num_rames):
        _, frame = video.read()
        frames[i] = frame
    video.release()
    return frames

def play(frames, stop_key=" "):
    for frame in frames:
        cv.imshow('Example', frame)
        if cv.waitKey(20) & 0xFF==ord(stop_key):
            break
    cv.destroyAllWindows()

In [4]:
video = cv.VideoCapture('./Examples/20200730/025Hz-01.avi')
nFrames = int(video.get(cv.CAP_PROP_FRAME_COUNT))
frames = [None] * nFrames
for i in range(nFrames):
    hasFrame, frame = video.read()
    frames[i] = frame

video.release()

for frame in frames:
    cv.imshow('Example', frame)
    if cv.waitKey(20) & 0xFF==ord(' '):
        break
    
cv.destroyAllWindows()


1602.0